## Simple Time Series - Univariate time series forecasting and make a single prediction

"univariate time series" refers to a time series that consists of single (scalar) observations recorded sequentially over equal time increments. 

In [ ]:
'''
Homework 7
Name: Austin Ramsey
ID: 000749074
LTU Honor Code: “I have neither given nor received any unauthorized aid in completing this work, nor have I presented someone else's work as my own.”

'''

"\nHomework 7\nName: Austin Ramsey\nID: 000749074\nLTU Honor Code: “I have neither given nor received any unauthorized aid in completing this work, nor have I presented someone else's work as my own.”\n\n"

In [2]:
# univariate LSTM, RNN example
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	#X, y = list(), list()
	X = []
	y = []
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
# for HW7
raw_seq = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 
           0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

# raw_seq = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# choose a number of time steps
n_steps = 4

# split into samples
X0, y = split_sequence(raw_seq, n_steps)
print(X0)
print(y)

[[0.   0.05 0.1  0.15]
 [0.05 0.1  0.15 0.2 ]
 [0.1  0.15 0.2  0.25]
 [0.15 0.2  0.25 0.3 ]
 [0.2  0.25 0.3  0.35]
 [0.25 0.3  0.35 0.4 ]
 [0.3  0.35 0.4  0.45]
 [0.35 0.4  0.45 0.5 ]
 [0.4  0.45 0.5  0.55]
 [0.45 0.5  0.55 0.6 ]
 [0.5  0.55 0.6  0.65]
 [0.55 0.6  0.65 0.7 ]
 [0.6  0.65 0.7  0.75]
 [0.65 0.7  0.75 0.8 ]
 [0.7  0.75 0.8  0.85]
 [0.75 0.8  0.85 0.9 ]]
[0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85
 0.9  0.95]


In [3]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X0.reshape((X0.shape[0], X0.shape[1], n_features))
print(X.shape)
#print(X)

(16, 4, 1)


#### DENSE Network

In [4]:
# DNN Model
from keras.models import Sequential
from keras.layers import Dense
model1 = Sequential()
model1.add(Dense(16, activation='sigmoid', input_shape=(n_steps,))) 

model1.add(Dense(8, activation='relu'))
model1.add(Dense(1))
model1.compile(optimizer='adam', loss='mse')
# fit model
print(X0.shape)
#print(X0)
model1.fit(X0, y, epochs=200, verbose=0)

(16, 4)


In [ ]:
model1.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 16)                80        
                                                                 
 dense_33 (Dense)            (None, 8)                 136       
                                                                 
 dense_34 (Dense)            (None, 1)                 9         
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# demonstrate prediction 1
x_input = array([0.80, 0.85, 0.90, 0.95])
x_input = x_input.reshape((1, n_steps))
yhat = model1.predict(x_input, verbose=0)
print(yhat)

[[0.80261445]]


#### RNN

In [ ]:
#print(X)
#print(n_features)

In [ ]:
# First RNN Model
from keras.layers import SimpleRNN, RNN, LSTM, GRU, Flatten, Reshape

model2 = Sequential()
model2.add(SimpleRNN(16, activation='tanh', input_shape=(n_steps,n_features)))
model2.add(Dense(8, activation="relu"))
model2.add(Dense(1))
model2.compile(optimizer='adam', loss='mse')
model2.fit(X, y, epochs=200, verbose=0) # fit model

In [ ]:
model2.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 16)                288       
                                                                 
 dense_35 (Dense)            (None, 8)                 136       
                                                                 
 dense_36 (Dense)            (None, 1)                 9         
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# demonstrate prediction 2
x_input = array([0.80, 0.85, 0.90, 0.95])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model2.predict(x_input, verbose=0)
print(yhat)

[[0.99204683]]


#### RNN*

In [45]:
# Second RNN Model with an additional RNN layer
from keras.layers import SimpleRNN, RNN, LSTM, GRU, Flatten, Reshape, Dropout

model3 = Sequential()
model3.add(SimpleRNN(16, activation='tanh', input_shape=(n_steps,n_features), return_sequences=True))
model3.add(Dropout(0.1))
model3.add(SimpleRNN(16, activation='tanh', return_sequences=False))
model3.add(Dense(8, activation='relu'))
model3.add(Dense(1))
model3.compile(optimizer='adam', loss='mse')
model3.fit(X, y, epochs=200, verbose=0) # fit model

In [46]:
model3.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_22 (SimpleRNN)   (None, 4, 16)             288       
                                                                 
 dropout_13 (Dropout)        (None, 4, 16)             0         
                                                                 
 simple_rnn_23 (SimpleRNN)   (None, 16)                528       
                                                                 
 dense_31 (Dense)            (None, 8)                 136       
                                                                 
 dense_32 (Dense)            (None, 1)                 9         
                                                                 
Total params: 961
Trainable params: 961
Non-trainable params: 0
_________________________________________________________________


In [47]:
# demonstrate prediction 3
x_input = array([0.80, 0.85, 0.90, 0.95])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model3.predict(x_input, verbose=0)
print(yhat)

[[0.9580602]]


#### LSTM

In [ ]:

from keras.layers import LSTM
# define model
model4 = Sequential()
model4.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model4.add(Dense(1))
model4.compile(optimizer='adam', loss='mse')
# fit model
model4.fit(X, y, epochs=200, verbose=0)

In [ ]:
# demonstrate prediction 4
x_input = array([0.80, 0.85, 0.90, 0.95])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model4.predict(x_input, verbose=0)
print(yhat)

[[1.0162028]]


### Analysis

After training and evaluating all of the models it can be seen that the LSTM model performed the best, whereas the DNN model performed the worst. The two RNN models performed very well, but the first RNN model performed slightly better. The second RNN model had an additional simpleRNN layer added and a dropout layer but this seemed to lower the performance slightly.